```sql
with event_case as (
select 'email_sent' as event_case union
select 'email_opened' union
select 'email_clicked' union
select 'other'
)

select
  emailaddress as email,

  date as event_at,
  process_date,

  case when event_case='other' then 
    case when lower(status) in ('rejected','deferred') then 'email_undeliverable' 
         when lower(status) = 'soft-bounced' then 'email_softbounce' 
         when lower(status) = 'bounced' then 'email_bounce' 
         when lower(status) = 'spam' then 'email_spam' end
  else event_case end as event_name,

  case when lower(subject) like '%transaction complete%' then 'transaction_complete'
     when lower(subject) like '%close%' then 'account_closure'
     when lower(subject) like '%initiated%' then 'transfer_initiated'
     when lower(subject) like '%dividends%' then 'dividends_paid'
     when lower(subject) like '%password%' then 'password_reset'
     when lower(subject) like '%factor%'then 'identity_verification'
     when lower(subject) like '%login%' then 'identity_verification'
     when lower(subject) like '%friend%' then 'referral_confirmation'
     when lower(subject) like '%direct deposit%' then 'direct_deposit_form'
     when lower(subject) like '%update%' then 'updated_account'
     else subject end as message_name,

  'mandrill' as message_source,
  'transactional' as message_type

from
  source_mandrill.activity,
  event_case

where
  event_case='email_sent' or
  event_case='email_clicked' and opens>=1 or
  event_case='email_opened' and clicks>=1 or
  event_case='other' and lower(status) in ('rejected','bounced','soft-bounced','deferred','spam')
```


# Original

```sql
WITH  mandrill_sent AS (

    SELECT
        u.uuid AS stash_user_uuid,
        CASE
            WHEN LOWER(m.subject) LIKE '%transaction complete%' THEN 'transaction_complete'
            WHEN LOWER(m.subject) LIKE '%close%' THEN 'account_closure'
            WHEN LOWER(m.subject) LIKE '%initiated%' THEN 'transfer_initiated'
            WHEN LOWER(m.subject) LIKE '%dividends%' THEN 'dividends_paid'
            WHEN LOWER(m.subject) LIKE '%password%' THEN 'password_reset'
            WHEN LOWER(m.subject) LIKE '%factor%' OR LOWER(m.subject) LIKE '%login%' THEN 'identity_verification'
            WHEN LOWER(m.subject) LIKE '%friend%' THEN 'referral_confirmation'
            WHEN LOWER(m.subject) LIKE '%direct deposit%' THEN 'direct_deposit_form'
            WHEN LOWER(m.subject) LIKE '%update%' THEN 'updated_account'
        ELSE subject END AS message_name,
        NULL AS message_id,
        NULL AS variant_name,
        NULL AS variant_id,
        NULL AS step_id,
        NULL AS step_name,
        'email_sent' AS event_name,
        m.date AS event_at,
        FALSE AS is_canvas,
        'mandrill' AS message_source,
        'transactional' AS message_type,
        NULL step_variant_id,
        process_date
    FROM {{   source('mandrill','activity')  }} m
    LEFT JOIN {{   source('monolith','users')  }} u ON m.emailaddress = u.email
    WHERE LOWER(m.subject) NOT LIKE '%test%' AND LOWER(m.subject) NOT LIKE '%staging%'


), mandrill_email_events AS (

    SELECT
        u.uuid AS stash_user_uuid,
        CASE
            WHEN LOWER(m.subject) LIKE '%transaction complete%' THEN 'transaction_complete'
            WHEN LOWER(m.subject) LIKE '%close%' THEN 'account_closure'
            WHEN LOWER(m.subject) LIKE '%initiated%' THEN 'transfer_initiated'
            WHEN LOWER(m.subject) LIKE '%dividends%' THEN 'dividends_paid'
            WHEN LOWER(m.subject) LIKE '%password%' THEN 'password_reset'
            WHEN LOWER(m.subject) LIKE '%factor%' OR LOWER(m.subject) LIKE '%login%' THEN 'identity_verification'
            WHEN LOWER(m.subject) LIKE '%friend%' THEN 'referral_confirmation'
            WHEN LOWER(m.subject) LIKE '%direct deposit%' THEN 'direct_deposit_form'
            WHEN LOWER(m.subject) LIKE '%update%' THEN 'updated_account'
            ELSE subject END AS message_name,
        CASE WHEN m.clicks >= 1 THEN TRUE ELSE FALSE END AS was_clicked,
        CASE WHEN m.opens >= 1 THEN TRUE ELSE FALSE END AS was_opened,
        m.status,
        m.date,
        process_date
    FROM {{   source('mandrill','activity')  }} m
    LEFT JOIN {{   source('monolith','users')  }} u ON m.emailaddress = u.email
    WHERE LOWER(m.subject) NOT LIKE '%test%' AND LOWER(m.subject) NOT LIKE '%staging%' 
        AND (m.clicks >= 1 or m.opens >=1)

), mandrill_open AS (

    SELECT
        stash_user_uuid,
        message_name,
        NULL AS message_id,
        NULL AS variant_name,
        NULL AS variant_id,
        NULL AS step_id,
        NULL AS step_name,
        'email_opened' AS event_name,
        m.date AS event_at,
        FALSE AS is_canvas,
        'mandrill' AS message_source,
        'transactional' AS message_type,
        NULL AS step_variant_id,
        process_date
    FROM mandrill_email_events m
    WHERE was_opened = TRUE 

), mandrill_click AS (

    SELECT
        stash_user_uuid,
        message_name,
        NULL AS message_id,
        NULL AS variant_name,
        NULL AS variant_id,
        NULL AS step_id,
        NULL AS step_name,
        'email_clicked' AS event_name,
        m.date AS event_at,
        FALSE AS is_canvas,
        'mandrill' AS message_source,
        'transactional' AS message_type,
        NULL AS step_variant_id,
        process_date
    FROM mandrill_email_events m
    WHERE was_clicked = TRUE 

), mandrill_undeliverable AS (

    SELECT
        stash_user_uuid,
        message_name,
        NULL AS message_id,
        NULL AS variant_name,
        NULL AS variant_id,
        NULL AS step_id,
        NULL AS step_name,
        CASE 
            WHEN LOWER(status) IN ('rejected','deferred') THEN 'email_undeliverable' 
            WHEN LOWER(status) = 'soft-bounced' THEN 'email_softbounce' 
            WHEN LOWER(status) = 'bounced' THEN 'email_bounce' 
            WHEN LOWER(status) = 'spam' THEN 'email_spam' 
        ELSE NULL END AS event_name,
        m.date AS event_at,
        FALSE AS is_canvas,
        'mandrill' AS message_source,
        'transactional' AS message_type,
        NULL AS step_variant_id,
        process_date
    FROM mandrill_email_events m
    WHERE LOWER(status) IN ('rejected','bounced','soft-bounced','deferred','spam')
```
